# ASSIGNMENT : IRIS MULTI-CLASS CLASSIFICATION

###### Purpose :
To predict the species of flower .
###### Description :
The dataset contains a set of 150 records under 5 attributes - Petal Length, Petal Width, Sepal Length, Sepal width and Class(Species).
###### Requirements :
1) Code must be in tf 2.0 .

2) Accuracy must be in between 95-97% .

3) Model shouldn't be Overfit (You can add drop out layer for this) .

### STEP 1 : Load all the necessary libraries 

https://www.kaggle.com/ash316/ml-from-scratch-with-iris
http://tekshinobi.com/analyzing-iris-dataset-with-keras-and-tensorflow-machine-learning-and-data-analysis/

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [3]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

### STEP 2 : Data Preparation
This step consists of multiple sub steps from data loading [download](https://github.com/ramsha275/PIAIC-Sir-Anees-Quarter-2/blob/master/Deep%20Learning/iris.csv),shuffling ,spliting in **Train** and **Test** sets to one-hot-enconding on labels . 


<h3>Create Dataframes for features and target</h3>

In [4]:
encoder = LabelBinarizer()
seed = 42

In [5]:
iris_data_df = pd.read_csv('iris.csv')
iris_data_df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [6]:
iris_data_df_1 = iris_data_df.loc[:, iris_data_df.columns != 'variety']
iris_data_df_1.head()

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [7]:
iris_target_df = pd.DataFrame({
    'Setosa' : iris_data_df['variety'] == 'Setosa',
    'Versicolor' : iris_data_df['variety'] == 'Versicolor',
    'Virginica' : iris_data_df['variety'] == 'Virginica',
})*1
iris_target_df.head()

,Setosa,Versicolor,Virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


<h3>Create training and testing datasets</h3>

In [8]:
X_train,X_test,y_train,y_test = train_test_split(iris_data_df_1,
                                                 iris_target_df,
                                                 test_size=0.30,
                                                 random_state=seed)

<h3>Feature scaling.</h3>

ML algos perform best when all dataset features have the same scale. Particularly, in Neural Nets, we use MinMaxScaler with range between 0 and 1.<br>
Also, since MinMaxScalar transformation makes us lose the column and index labels, we need to recreate the dataframes for scaled data and target.

In [9]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler

MinMaxScaler(copy=True, feature_range=(0, 1))

In [10]:
X_train = pd.DataFrame(scaler.fit_transform(X_train),
                               columns=X_train.columns,
                               index=X_train.index)
X_train.head()

,sepal.length,sepal.width,petal.length,petal.width
81,0.352941,0.181818,0.464286,0.375000
133,0.588235,0.363636,0.714286,0.583333
137,0.617647,0.500000,0.785714,0.708333
75,0.676471,0.454545,0.589286,0.541667
109,0.852941,0.727273,0.892857,1.000000


In [11]:
X_test = pd.DataFrame(scaler.transform(X_test),
                           columns=X_test.columns,
                           index=X_test.index)
X_test.head()

,sepal.length,sepal.width,petal.length,petal.width
73,0.529412,0.363636,0.642857,0.458333
18,0.411765,0.818182,0.107143,0.083333
118,1.000000,0.272727,1.035714,0.916667
78,0.500000,0.409091,0.607143,0.583333
76,0.735294,0.363636,0.660714,0.541667


<h5>Optional micro detail:</h5>
Note that we use <b>fit_transform</b> on X_train and <b>transform</b> on X_test.<br>
It is very important that we don’t fit on test data.

### STEP 3 : Model Architecture 


###### Input : 4 
###### 1 hidden Layer : 8 nodes
###### Output : 3

<h3>Build Model </h3>

In [12]:
model = Sequential()
model.add(Dense(8, input_dim=4, activation='tanh', name='input_layer'))
model.add(Dense(8, activation='tanh', name='hidden_layer'))
model.add(Dense(3, activation='softmax', name='output_layer'))
model

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### STEP 4 : Compilation Step 

In [13]:
model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])
model

<h5>Optional micro detail:</h5>
<ol>
    <li>
        Input layer(layer_1) has 4 inputs corresponding to 4 feature columns in X_train. <br> The output_layer has 3 outputs for the three classes in target. </li>
    <li>
        You can try different activation functions in layer_1, layer_2 and layer_3 like sigmoid or relu. <br>Sometimes choice of activation function affects the results a great deal. In most cases, you will be using non-linear activation functions like tanh, sigmoid or relu.</li>
    <li>
        <b>metrics=[‘accuracy’]:</b> Here default accuracy is used which is categorical_accuracy. Some other accuracy settings are binary_accuracy, sparse_categorical_accuracy etc. Default is the most appropriate here. </li>
    </ol>

<h3>Create estimator</h3>

In [14]:
estimator = KerasClassifier(
        build_fn=model,
        epochs=200, batch_size=20,
        verbose=2)
estimator

<h5>Optional micro detail:</h5>
Note that setting verbose=2 will give us some helpful messages during training. If you don’t want to see these messages, set verbose=0 to turn off these messages.<br>
The debug messages will show two metrics: loss and accuracy. loss refers to categorical_crossentropy loss function used in the model. accuracy refers to the default accuracy metric which is categorical_accuracy’

<h3>Do the Model cross validation</h3>

Since the dataset is quite small, we can get away with just 5 fold cross validation.
On bigger datasets, increase the folds by modifying <b>n_splits</b>

In [15]:
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)
kfold

KFold(n_splits=5, random_state=42, shuffle=True)

In [19]:
results = cross_val_score(estimator, X_train, y_train, cv=kfold)

TypeError: can't pickle _thread.RLock objects

In [ ]:
print("Model Performance: mean: %.2f%% std: (%.2f%%)" % (results.mean()*100, results.std()*100))

<h2>Train The Model</h2>

In [14]:
model.fit(
       X_train,
       y_train,
       epochs=200,
       shuffle=True, # shuffle data randomly.
       #NNs perform best on randomly shuffled data
       verbose=2 # this will tell keras to print more detailed info
       # during trainnig to know what is going on
       )

Epoch 1/200
105/105 - 1s - loss: 1.3040 - acc: 0.3429
Epoch 2/200
105/105 - 0s - loss: 1.2884 - acc: 0.3429
Epoch 3/200
105/105 - 0s - loss: 1.2723 - acc: 0.3429
Epoch 4/200
105/105 - 0s - loss: 1.2583 - acc: 0.3429
Epoch 5/200
105/105 - 0s - loss: 1.2450 - acc: 0.3429
Epoch 6/200
105/105 - 0s - loss: 1.2306 - acc: 0.3429
Epoch 7/200
105/105 - 0s - loss: 1.2170 - acc: 0.3429
Epoch 8/200
105/105 - 0s - loss: 1.2053 - acc: 0.3429
Epoch 9/200
105/105 - 0s - loss: 1.1928 - acc: 0.3524
Epoch 10/200
105/105 - 0s - loss: 1.1803 - acc: 0.3524
Epoch 11/200
105/105 - 0s - loss: 1.1695 - acc: 0.3524
Epoch 12/200
105/105 - 0s - loss: 1.1585 - acc: 0.3524
Epoch 13/200
105/105 - 0s - loss: 1.1483 - acc: 0.3524
Epoch 14/200
105/105 - 0s - loss: 1.1384 - acc: 0.3619
Epoch 15/200
105/105 - 0s - loss: 1.1289 - acc: 0.3619
Epoch 16/200
105/105 - 0s - loss: 1.1198 - acc: 0.3619
Epoch 17/200
105/105 - 0s - loss: 1.1111 - acc: 0.3619
Epoch 18/200
105/105 - 0s - loss: 1.1021 - acc: 0.3619
Epoch 19/200
105/10

Epoch 150/200
105/105 - 0s - loss: 0.5518 - acc: 0.8095
Epoch 151/200
105/105 - 0s - loss: 0.5496 - acc: 0.8190
Epoch 152/200
105/105 - 0s - loss: 0.5474 - acc: 0.8286
Epoch 153/200
105/105 - 0s - loss: 0.5451 - acc: 0.8286
Epoch 154/200
105/105 - 0s - loss: 0.5430 - acc: 0.8286
Epoch 155/200
105/105 - 0s - loss: 0.5409 - acc: 0.8381
Epoch 156/200
105/105 - 0s - loss: 0.5389 - acc: 0.8381
Epoch 157/200
105/105 - 0s - loss: 0.5368 - acc: 0.8476
Epoch 158/200
105/105 - 0s - loss: 0.5346 - acc: 0.8476
Epoch 159/200
105/105 - 0s - loss: 0.5328 - acc: 0.8381
Epoch 160/200
105/105 - 0s - loss: 0.5307 - acc: 0.8286
Epoch 161/200
105/105 - 0s - loss: 0.5286 - acc: 0.8381
Epoch 162/200
105/105 - 0s - loss: 0.5266 - acc: 0.8476
Epoch 163/200
105/105 - 0s - loss: 0.5252 - acc: 0.8286
Epoch 164/200
105/105 - 0s - loss: 0.5228 - acc: 0.8286
Epoch 165/200
105/105 - 0s - loss: 0.5210 - acc: 0.8286
Epoch 166/200
105/105 - 0s - loss: 0.5191 - acc: 0.8381
Epoch 167/200
105/105 - 0s - loss: 0.5172 - acc:

### STEP 5 : Evaluation 

In [15]:
test_error_rate = model.evaluate(X_test, y_test, verbose=2)
test_error_rate

45/45 - 0s - loss: 0.4007 - acc: 0.8667


[0.40068522294362385, 0.8666667]

In [16]:
print(
      "{} : {:.2f}%".format(model.metrics_names[1],
              test_error_rate[1]*100))
print(
      "{} : {:.2f}%".format(model.metrics_names[0],
              test_error_rate[0]*100))
#You can run the code again to see how the model perform

acc : 86.67%
loss : 40.07%


### STEP 6 : Prediction 

In [17]:
predicted_targets = model.predict_classes(X_test)
predicted_targets

array([1, 0, 2, 1, 2, 0, 1, 2, 2, 1, 2, 0, 0, 0, 0, 2, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 2, 1, 2, 0,
       0], dtype=int64)

In [18]:
iris_target_array = iris_target_df.values
#iris_target_array

In [19]:
target = encoder.fit_transform(iris_target_array)
#target

In [20]:
true_targets = encoder.inverse_transform(y_test.values)
#true_targets

In [21]:
#performance_tracker(predicted_targets, true_targets)
flowers = {0:'Setosa', 1:'Versicolor', 2:'Virginica'}
print("Flowers in test set: Setosa={} Versicolor={} Virginica={}".format(
            y_test.Setosa.sum(), y_test.Versicolor.sum(),
            y_test.Virginica.sum()))

Flowers in test set: Setosa=19 Versicolor=13 Virginica=13


In [22]:
for act,exp in zip(predicted_targets, true_targets):
        tup = np.where(exp == 1)
        if act != tup[0][0]:
            print("ERROR: {} predicted as {}".format(flowers[tup[0][0]],
                  flowers[act]))
        else:
            print("CORRECT: {} predicted as {}".format(flowers[tup[0][0]],
                  flowers[act]))

CORRECT: Versicolor predicted as Versicolor
CORRECT: Setosa predicted as Setosa
CORRECT: Virginica predicted as Virginica
CORRECT: Versicolor predicted as Versicolor
ERROR: Versicolor predicted as Virginica
CORRECT: Setosa predicted as Setosa
CORRECT: Versicolor predicted as Versicolor
CORRECT: Virginica predicted as Virginica
ERROR: Versicolor predicted as Virginica
CORRECT: Versicolor predicted as Versicolor
CORRECT: Virginica predicted as Virginica
CORRECT: Setosa predicted as Setosa
CORRECT: Setosa predicted as Setosa
CORRECT: Setosa predicted as Setosa
CORRECT: Setosa predicted as Setosa
ERROR: Versicolor predicted as Virginica
CORRECT: Virginica predicted as Virginica
CORRECT: Versicolor predicted as Versicolor
CORRECT: Versicolor predicted as Versicolor
CORRECT: Virginica predicted as Virginica
CORRECT: Setosa predicted as Setosa
CORRECT: Virginica predicted as Virginica
CORRECT: Setosa predicted as Setosa
CORRECT: Virginica predicted as Virginica
CORRECT: Virginica predicted as